In [23]:
import sys
from pygame import K_UP, K_DOWN, RLEACCEL, QUIT, init
from pygame.display import set_mode, set_caption, flip
from pygame.event import get
from pygame.font import Font
from pygame.image import load
from pygame.key import get_pressed
from pygame.mixer import Sound
from pygame.sprite import Sprite, collide_rect
from pygame.time import Clock
import pygame


ALTO = 480
ANCHO = 640


def cargar_imagen(nombre, transparent=False):
    """
    Carga la imagen dada su ubicacion NOMBRE, además aplica
    transparencias si se le indica. Devuelve un objeto de tipo
    |pygame.Surface|
    """
    try:
        imagen = load(nombre)
    except Exception as message:
        raise SystemExit(message)
    imagen = imagen.convert()
    if transparent:
        color = imagen.get_at((0, 0))
        imagen.set_colorkey(color, RLEACCEL)
    return imagen


def texto(texto, posicion_x, posicion_y, color=(255, 255, 255)):
    """
    Método para insertar el texto TEXTO en la posición (POSICION_X,
    POSICION_Y) con el color COLOR
    """
    fuente = Font('17-years-ago.ttf', 25)
    salida = Font.render(fuente, texto, 1, color)
    salida_rect = salida.get_rect()
    salida_rect.centerx = posicion_x
    salida_rect.centery = posicion_y
    return salida, salida_rect


class Pala(Sprite):

    def __init__(self, centerx, speed = 1, imagen = "pala.png"):
        super().__init__()
        self.image = cargar_imagen(imagen)
        self.rect = self.image.get_rect()
        self.rect.centerx = centerx
        self.rect.centery = ALTO / 2
        self.speed = speed

    def mover(self, time, keys):
        """Determina la posición de la pala en el tiempo TIME siempre y
        cuando KEYS sea igual a las teclas UP y DOWN."""
        if self.rect.top >= 0:
            if keys[K_UP]:
                self.rect.centery -= self.speed * time
        if self.rect.bottom <= ALTO:
            if keys[K_DOWN]:
                self.rect.centery += self.speed * time

    def ia(self, time, ball):
        """Inteligencia artifical modelada a través de un agente
        reactivo. Dependiendo de la posición de la pelota BALL va a
        realizar su movimiento para contrarrestar el envio del
        oponente. El parámetro TIME cual es la cantidad de pixeles que
        se debe mover."""
        if ball.speed[0] >= 0 and ball.rect.centerx >= 0:
            if self.rect.centery < ball.rect.centery:
                self.rect.centery += self.speed * time
            if self.rect.centery > ball.rect.centery:
                self.rect.centery -= self.speed * time


class Bola(Sprite):
    """
    Clase Bola: hereda atributos directamente de la clase Sprite, que
    se encuentra en el modulo pygame.sprite. Permite modelar la pelota
    con la que se interactua en el juego.
    """

    def __init__(self):
        """
        Constructor que inicializa los siguientes valores:
        - La imagen a desplegar.
        - Posición, la cual es representada a través de un rectangulo
          invisible.
        - Velocidad por defecto de la pelota en el eje Y y Y
        """
        Sprite.__init__(self)
        self.image = cargar_imagen("pelota.png", True)
        self.rect = self.image.get_rect()
        self.rect.centerx = ANCHO / 2
        self.rect.centery = ALTO / 2
        self.sound = Sound('rebote.wav')
        self.speed = [0.3, 0.3]

    def actualizar(self, time, pala, pala_cpu, puntos):
        """
        Actualiza la posición de la pelota en tantos pixeles
        determinado por TIME. Además permite identificar si interactua
        con PALA y PALA_CPU. En caso de que existan puntos, actualiza
        los valores de PUNTOS.
        """
        self.rect.centerx += self.speed[0] * time
        self.rect.centery += self.speed[1] * time
        if self.rect.left <= 0:
            puntos[1] += 1
        if self.rect.right >= ANCHO:
            puntos[0] += 1
        if self.rect.left <= 0 or self.rect.right >= ANCHO:
            self.speed[0] = -self.speed[0]
            self.rect.centerx += self.speed[0] * time
        if self.rect.top <= 0 or self.rect.bottom >= ALTO:
            self.speed[1] = -self.speed[1]
            self.rect.centery += self.speed[1] * time
        if self.rect.left == 0 or self.rect.right == ANCHO:
            self.sound.play()
        if self.rect.top == 0 or self.rect.bottom == ALTO:
            self.sound.play()
        if collide_rect(self, pala):
            self.speed[0] = -self.speed[0]
            self.rect.centerx += self.speed[0] * time
            self.sound.play()
        if collide_rect(self, pala_cpu):
            self.speed[0] = -self.speed[0]
            self.rect.centerx += self.speed[0] * time
            self.sound.play()
        
        return puntos

In [24]:
from neo4j import GraphDatabase
from tkinter import *
import tkinter.font as tkFont
cuenta=0

root = Tk()
root.title("LOGIN IA")
root.geometry('400x400')
root.resizable(width=0, height=0)
fontStyle = tkFont.Font(family="Lucida Grande", size=12)
fontStyle1 = tkFont.Font(family="Lucida Grande", size=30)
fontStyle2 = tkFont.Font(family="Lucida Grande", size=15)

eusuario=Label(root,text='LOGIN', font=fontStyle1)
eusuario.place(x=150,y=20)

eusuario=Label(root,text='Usuario', font=fontStyle)
eusuario.place(x=50,y=100)
usuario = Entry(root, fg='black', bg='white', justify='center')
usuario.place(x=110, y=100, width=200)           

eedad=Label(root,text='Edad: ', font=fontStyle)
eedad.place(x=50,y=150)
edad = Entry(root, fg='black', bg='white', justify='center')
edad.place(x=110, y=150, width=100)

epais=Label(root,text='Ciudad/Pais: ', font=fontStyle)
epais.place(x=50,y=200)
pais = Entry(root, fg='black', bg='white', justify='center')
pais.place(x=110, y=200, width=200)

ecorreo=Label(root,text='correo', font=fontStyle)
ecorreo.place(x=50,y=250)
correo = Entry(root, fg='black', bg='white', justify='center')
correo.place(x=110, y=250, width=200) 

def perdiste():
    frame2= Frame(root, width=200, height=100, bg='black')
    frame2.place(x=10, y=100)
    
    eedad=Label(frame2,text='PERDISTE: ', font=fontStyle)
    eedad.place(x=3,y=7)
    
class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close() 
        
    def crear_usuario(self, tx, nombre):
        tx.run("CREATE (Usuario:Jugador {usuario: $nombre})", nombre=nombre)
        
    def crear_usuarioj(self, tx, usr):
        tx.run("CREATE (Usr:Jugador {nombre: $usr})", usr=usr)
    
    def crear_edad(self, tx, edad):
        tx.run("CREATE (Edad:Jugador {edad: $edad})", edad=edad)
        
    def crear_pais(self, tx, pais):
        tx.run("CREATE (Pais:Jugador {pais: $pais})", pais=pais)

    def crear_correo(self, tx, correo):
        tx.run("CREATE (Correo:Jugador {correo: $correo})", correo=correo)
    
    def crear_puntaje(self, tx, puntaje):
        tx.run("CREATE (Puntaje:Jugador {puntaje: $puntaje})", puntaje=puntaje)
        
    def crear_puntajep(self, tx, puntajeia):
        tx.run("CREATE (Puntajeia:Jugador {puntajeia: $puntajeia})", puntajeia=puntajeia)
        
        
    def crear_relacion_usuario_usr(self, tx, usuario, usr):
        tx.run("MATCH (a:Jugador {usuario: $usuario}) "
               "MATCH (b:Jugador {nombre: $usr}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, usr=usr)    
        
    def crear_relacion_usuario_edad(self, tx, usuario, edad):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {edad: $edad}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, edad=edad)
    
    def crear_relacion_usuario_pais(self, tx, usuario, pais):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {pais: $pais}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, pais=pais)
        
    def crear_relacion_usuario_correo(self, tx, usuario, correo):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {correo: $correo}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, correo=correo)
    
    def crear_relacion_usuario_puntaje(self, tx, usuario, puntaje):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {puntaje: $puntaje}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, puntaje=puntaje)
        
    def crear_relacion_usuario_puntajeia(self, tx, usuario, puntajeia):
        tx.run("MATCH (a:Jugador {nombre: $usuario}) "
               "MATCH (b:Jugador {puntajeia: $puntajeia}) "
               "MERGE (a)-[r:relacion]->(b)",
               usuario=usuario, puntajeia=puntajeia)
        
    def grafica(self,tx):
        tx.run("CALL gds.graph.create(\n"
                "'vecinos',\n"
                "{\n"
                    "Person: {\n"
                        "label: 'Person',\n"
                        "properties: 'age'\n"
                    "}\n"
                "},\n"
                "'*'\n"
            ");")
        
    def cercanos(self, tx):
        result=tx.run("CALL gds.beta.knn.stream('vecinos', {\n"
                    "topK: 4,\n"
                    "nodeWeightProperty: 'puntajeia',\n"
                    "randomSeed: 42,\n"
                    "concurrency: 1,\n"
                    "sampleRate: 1.0,\n"
                    "deltaThreshold: 0.0\n"
                "})\n"
                "YIELD node1, node2, similarity\n"
                "RETURN gds.util.asNode(node1).puntajeia AS Person1, gds.util.asNode(node2).puntajeia AS Person2, similarity\n"
                "ORDER BY similarity , Person1, Person2")
        for line in result:
            r1=(line["similarity"])
            r2=(line["Person1"])
            r3=(line["Person2"])
            print(r1,"   ",r2,"   ",r3)
            
def contador():
    global cuenta
    cuenta += 1    
    
def vecinoscercanos():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'proyecto')
    with neo4j._driver.session() as session:
        session.read_transaction(neo4j.cercanos)
        
def game_loop():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'proyecto')
    usr=str(usuario.get())
    ed=str(edad.get())
    pa=str(pais.get())
    mail=str(correo.get())
    
    with neo4j._driver.session() as session:

        session.write_transaction(neo4j.crear_usuarioj , usr)
    session.write_transaction(neo4j.crear_edad , ed)
    session.write_transaction(neo4j.crear_pais , pa)
    session.write_transaction(neo4j.crear_correo , mail)

    session.write_transaction(neo4j.crear_relacion_usuario_usr,"Usuario",usr)
    session.write_transaction(neo4j.crear_relacion_usuario_edad,usr,ed)
    session.write_transaction(neo4j.crear_relacion_usuario_pais,usr,pa)
    session.write_transaction(neo4j.crear_relacion_usuario_correo,usr,mail)
    
    """Ciclo de juego. Mientras no se cierre la ventana, el juego no
    termina."""
    pantalla = set_mode((ANCHO, ALTO))
    set_caption("Pong <3")
    fondo = cargar_imagen('fondo_pong.png')
    bola = Bola()
    pala = Pala(5, imagen = "pala.png")
    pala_cpu = Pala(ANCHO - 5, 0.3, "pala.png")
    clock = Clock()
    puntos = [0, 0]
    sonido = Sound('Tetris.ogg')
    sonido.play()
    while puntos[0] <= 10 and puntos[1] <= 10:
        time = clock.tick(60)
        keys = get_pressed()
        if(puntos[1]==10):
            puntosg=int(puntos[0])
            puntosp=int(puntos[1])
            session.write_transaction(neo4j.crear_puntaje , puntosg)
            session.write_transaction(neo4j.crear_relacion_usuario_puntaje,usr,puntosg)
                
            session.write_transaction(neo4j.crear_puntajep , puntosp)
            session.write_transaction(neo4j.crear_relacion_usuario_puntajeia,usr,puntosp)
            
            perdiste()
            sonido.stop()
            pygame.quit()
            
            
        for eventos in get():
            if eventos.type == QUIT:
                puntosg=int(puntos[0])
                puntosp=int(puntos[1])
                session.write_transaction(neo4j.crear_puntaje , puntosg)
                session.write_transaction(neo4j.crear_relacion_usuario_puntaje,usr,puntosg)
                
                session.write_transaction(neo4j.crear_puntajep , puntosp)
                session.write_transaction(neo4j.crear_relacion_usuario_puntajeia,usr,puntosp)
                print(puntos[1])
                sonido.stop()
                pygame.quit()
        bola.actualizar(time, pala, pala_cpu, puntos)
        pala.mover(time, keys)
        pala_cpu.ia(time, bola)
        puntos_jug, puntos_jug_rect = texto(
            "TU: " + str(puntos[0]), 140, 450)
        puntos_cpu, puntos_cpu_rect = texto(
            "IA: " + str(puntos[1]), ANCHO - ANCHO / 4, 450)
        pantalla.blit(fondo, (0, 0))
        pantalla.blit(bola.image, bola.rect)
        pantalla.blit(pala.image, pala.rect)
        pantalla.blit(pala_cpu.image, pala_cpu.rect)
        pantalla.blit(puntos_jug, puntos_jug_rect)
        pantalla.blit(puntos_cpu, puntos_cpu_rect)
        flip()
    
    
    
def jugar():
    if __name__ == '__main__':
        init()
        game_loop()
        

    
    
btn1 = Button(root, text="Ingresar", bg="MediumAquamarine", fg="white", height = 2, width = 15,font=fontStyle2,command=jugar)
btn1.place(x=120, y=300)
root.mainloop()

2


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/RobertoPacho/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-24-49c6219fdbfd>", line 229, in jugar
    game_loop()
  File "<ipython-input-24-49c6219fdbfd>", line 212, in game_loop
    puntos_jug, puntos_jug_rect = texto(
  File "<ipython-input-23-c036391dfb87>", line 40, in texto
    fuente = Font('17-years-ago.ttf', 25)
pygame.error: font not initialized


#/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/grass_block.png


In [22]:
vecinoscercanos()

1.0842021724855044e-19     None     None
1.0842021724855044e-19     None     None
1.0842021724855044e-19     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     None
1.0     None     No

TclError: can't invoke "frame" command: application has been destroyed